# 项目背景及要求

优衣库（英文名称：UNIQLO，日文假名发音：ユニクロ），为日本迅销公司的核心品牌,建立于1984年，当年是一家销售西服的小服装店，现已成为国际知名服装品牌。优衣库现任董事长兼总经理柳井正在日本首次引进了大卖场式的服装销售方式，通过独特的商品策划、开发和销售体系来实现店铺运作的低成本化，由此引发了优衣库的热卖潮。

优衣库(Uniqlo) 的内在涵义是指通过摒弃了不必要装潢装饰的仓储型店铺，采用超市型的自助购物方式，以合理可信的价格提供顾客希望的商品价廉物美的休闲装“UNIQLO”是Unique Clothing Warehouse的缩写，意为消费者提供“低价良品、品质保证”的经营理念，在日本经济低迷时期取得了惊人的业绩。

根据提供的数据，利用Python进行可视化，并回答如下问题：


-     整体销售情况随着时间的变化是怎样的？
-    不同产品的销售情况是怎样的？顾客偏爱哪一种购买方式？
- 销售额和产品成本之间的关系怎么样？


# 项目代码

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import seaborn as sns
sns.set(style="whitegrid")

In [3]:
#windows
plt.rcParams['font.sans-serif'] = ['simhei']

In [ ]:
#os
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

## 读取数据

In [11]:
df_origin = pd.read_csv('./data/优衣库数据.csv',engine='python',encoding='utf-8')

伪代码：
-

In [13]:
df_origin.shape

(22293, 1)

In [14]:
df_origin.columns 

Index(['store_id'], dtype='object')

In [12]:
df_origin.head()

store_id
658 深圳 线下 Female 25-29 Weekday 当季新品 4 796.0 4 4        59
146 杭州 线下 Female 25-29 Weekday 运动   1 149.0 1 1        49
70  深圳 线下 Male   >=60  Weekday T恤   2 178.0 2 2        49
658 深圳 线下 Female 25-29 Weekday T恤   1 59.0  1 1        49
229 深圳 线下 Male   20-24 Weekend 袜子   2 65.0  2 3         9

## EDA

In [ ]:
df_origin.info()

In [ ]:
df_origin.describe(include='all')

- 处理异常值

营收存在异常值。

In [ ]:
df_origin['revenue'].plot(kind='hist',logx=True,bins=1000);

In [ ]:
df_origin[df_origin['revenue']<0]

In [ ]:
df_clean = df_origin.copy()

In [ ]:
#删除小于零的这条
drop_idx = df_clean[df_clean['revenue']<0].index
df_clean = df_clean.drop(drop_idx)

- store_id应该是str类型

In [ ]:
df_clean['store_id'] = df_clean['store_id'].astype(str)

- 增加计算字段
    - 人均订单量`order/customer`
    - 人均购买商品量`quant/customer`
    - 订单平均包含商品量`quant/order`
    - 商品平均销售额`revenue/quant`
    - 人均消费额`revenue/customer`
    - 利润`商品平均销售额-unit_cost`

In [ ]:
df_clean['uni_order_num'] = df_clean['order']/df_clean['customer']
df_clean['uni_quant_num'] = df_clean['quant']/df_clean['customer']
df_clean['uni_quant_of_order'] = df_clean['quant']/df_clean['order']
df_clean['uni_revenue_of_product'] = df_clean['revenue']/df_clean['quant']
df_clean['uni_revenue_of_customer'] = df_clean['revenue']/df_clean['customer']
df_clean['margin'] = df_clean['uni_revenue_of_product'] - df_clean['unit_cost']

数据概况：
- 数据包含7列分类变量和10列数值变量；
- 各变量分布情况如下：

In [ ]:
df_clean.describe(include='all')

### 分类变量

In [ ]:
def cat_bar_plot(col_name,order=None):
    value_count_df = pd.DataFrame(df_clean[col_name].value_counts()).reset_index()
    value_count_df = value_count_df.rename(columns={'index':col_name,col_name:'count'})
    g = sns.catplot(x=f'{col_name}', y="count",data=value_count_df,height=6, kind="bar", palette="muted",order=order)
    g.despine(left=True)
    g.set_ylabels('Count of Categories in {}'.format(col_name))

In [ ]:
age_orders = ['<20','20-24','25-29',  '30-34','35-39',  '40-44','45-49', '50-54','55-59','>=60', 'Unkown']
for col in df_clean.columns[1:7]:
    order = None
    if col == 'age_group':
        order = age_orders
    cat_bar_plot(col,order=order)

结论：
- 数据主要集中在深圳、杭州、武汉三所城市
- 选择线下购买的客户远大于线上购买
- 客户中女性数量多于男性近一倍
- 客户的年龄段主要集中在20-40岁之间，此外，还有一些用户（60岁以上）可疑填写不实信息
- 客户选择周内购买的总次数要稍大于周末购买总次数，但如果按日平均下来，周末单日的次数要多于周内单日
- T恤是最畅销的产品

### 数值变量

In [ ]:
fig,axes = plt.subplots(11,1,figsize=(5,50))
i = 0
for col in df_clean.columns[7:]:
    logx = False
    if col in ['revenue','uni_revenue_of_customer']:
        logx = True
    df_clean[col].plot(kind='hist',logx=logx,title=f'Distribution of {col}',ax=axes[i])
    i += 1

In [ ]:
sns.heatmap(df_clean.iloc[:,7:].corr());

- 利润主要与单位商品销售额、单位顾客消费额及总销售额有关；
- 顾客数量、总销售额、订单数及商品数两两均呈强正相关关系。

## 项目问题

### 问题一：整体销售情况随着时间的变化是怎样的？

数据中涉及到时间的只有变量`wkd_ind`，该变量中只包含了`weekday`和`weekend`两类变量，所以我们可以从渠道、性别、顾客数量、销售额、销售产品数量及利润等维度对其进行对比分析及可视化。

In [ ]:
def cat_wkd_ind_plot(col,figsize=(7,7)):
    '''
    绘制变量wkd_ind与其他分类变量之间的统计比例柱状图
    '''
    tmp = df_clean.groupby(col)['wkd_ind'].apply(lambda x: x.value_counts()/x.count())
    tmp = tmp.reset_index().rename(columns={'level_1':'wkd_ind','wkd_ind':'rate'})
    plt.figure(figsize=figsize)
    g = sns.barplot(x=col,y='rate',hue='wkd_ind',data=tmp)
    for index,row in tmp.iterrows():
        #在柱状图上绘制该类别的数量
        if row.name%2 == 0:
            g.text(row.name//2-0.2,row.rate,round(row.rate,2),color="black",ha="center")
        else:
            g.text(row.name//2+0.2,row.rate,round(row.rate,2),color="black",ha="center")

- 线上还是线下

In [ ]:
cat_wkd_ind_plot('channel')

不管是周内还是周末，选择线上和线下的客户比例相当。

- 男女是否有区别

In [ ]:
cat_wkd_ind_plot('gender_group')

相比于女生，男生有稍高的比例(+4%)选择在周末购买。

- 不同年龄段的影响

In [ ]:
cat_wkd_ind_plot('age_group',figsize=(14,7))

24岁以下的两组比例相当，25岁以上组(>=60分组不予考虑)随着年龄的逐渐增长，在周内购物的比例也逐渐升高。

- 销售额等变量的对比

In [ ]:
def num_wkd_ind_plot(method='sum'):
    '''
    绘制数值型变量与时间的关系对比比例图。
    '''
    numerical_cols = ['customer', 'revenue', 'order', 'quant', 'unit_cost','uni_order_num', 'uni_quant_num', 'uni_quant_of_order',
                      'uni_revenue_of_product', 'uni_revenue_of_customer', 'margin']
    trans_to_chinese = ['顾客量','总收入','订单量','商品数','单位成本','人均\n订单量','人均\n商品量','订单均\n含商品量','单位收入',
                        '人均收入','单位利润']
    trans_dict = dict(zip(numerical_cols,trans_to_chinese))
    
    
    tmp = df_clean.groupby('wkd_ind')[numerical_cols].apply(eval(method))
    tmp.loc['Weekend'] = tmp.loc['Weekend']/tmp.loc['Weekday']
    tmp.loc['Weekday'] = tmp.loc['Weekday']/tmp.loc['Weekday']
    tmp = tmp.reset_index()
    tmp = tmp.melt(id_vars='wkd_ind',value_vars=numerical_cols)
    tmp['variable'] = tmp['variable'].map(trans_dict)
    
    plt.figure(figsize=(10,5))
    sns.barplot(x='variable',y='value',hue='wkd_ind',data=tmp)

In [ ]:
#总量
num_wkd_ind_plot()

In [ ]:
#均值
num_wkd_ind_plot(method='np.mean')

In [ ]:
#标准差
num_wkd_ind_plot(method='np.std')

三幅图都是以周内总量作为分母，求得的占比图。
- 就总量来看，周末两天的总量一直低于周内五天的总量
- 就均值来看，周末具有较高的单位成本，但也带来了更多的单位收入和单位利润
- 就标准差来看，周末的单位收入和单位利润更不稳定

### 问题二：不同产品的销售情况是怎样的？顾客偏爱哪一种购买方式？

- 不同产品即指product字段中不同类别的产品，销售情况即为销售额revenue，可生成柱状图进行可视化
- 购买方式只有channel是线上还是线下这一个指标，而顾客可以从不同性别gender_group、年龄段age_group、城市city三个维度进行分解，因此本问即为探究不同性别、年龄段和城市的顾客对线上、线下两种购买方式的偏好，可生成柱状图进行可视化的呈现

In [ ]:
#总销售额
df_clean.groupby('product')['revenue'].sum().sort_values(ascending=False).plot(kind='bar');

In [ ]:
#总销售数量
df_clean.groupby('product')['quant'].sum().sort_values(ascending=False).plot(kind='bar');

In [ ]:
#利润
df_clean.groupby('product')['margin'].sum().sort_values(ascending=False).plot(kind='bar');

In [ ]:
#利润均值
df_clean.groupby('product')['margin'].mean().sort_values(ascending=False).plot(kind='bar');

- T恤占绝对领先地位，在销售额、销售数量和利润上都遥遥领先；当季新品和配件占据第二、三名，其中当季新品的销售额和数量较配件高，但利润较配件低；
- 毛衣、裙子和配件的平均利润最高，都在80元左右。

In [ ]:
def channel_plot(col):
    '''
    绘制与渠道相关的分类比例图
    '''
    tmp = df_clean.groupby(col)['channel'].apply(lambda x:x.value_counts()/x.count())
    tmp = pd.DataFrame(tmp).reset_index()
    tmp = tmp.rename(columns={'level_1':'channel','channel':'rate'})
    sns.barplot(x=col,y='rate',hue='channel',data=tmp);

In [ ]:
channel_plot('city')

In [ ]:
channel_plot('gender_group')

In [ ]:
channel_plot('age_group')

- 广州的顾客更偏爱在线上购买，上海、西安和重庆也有一定的线上购买比例，其余城市均为在线下购买；
- 不同性别在选择购买渠道时比例相当；
- 就年龄段来看，所有年龄段选择线上购买的比例相当，都在15%-20%之间，其中25-29岁选择线上比例最高，55-59岁选择线上比例最低。

### 问题三：销售额和产品成本之间的关系怎么样？

-   思路一：单件产品销售额-成本为利润margin，margin是如何分布的？是否存在亏本销售的产品？
-    思路二：探究实际销售额和产品成本之间的关系，即为求它们之间的相关，若成正相关，则产品成本越高，销售额越高，或许为高端商品；若成负相关，则成本越低，销售额越高，为薄利多销的模式。
    - 还可以拆分得更细，探究不同城市和门店中成本和销售额的相关性。

In [ ]:
df_clean['margin'].plot(kind='hist',bins=15);

- 利润主要集中在0-75元之间；
- 存在一定量的亏本（即利润小于0）的产品。

In [ ]:
#聚焦亏本产品
df_loss = df_clean[df_clean['margin']<0]

In [ ]:
#城市
(df_loss['city'].value_counts()/df_clean['city'].value_counts()).sort_values().plot(kind='bar');

In [ ]:
df_loss.columns

In [ ]:
#渠道
(df_loss['channel'].value_counts()/df_clean['channel'].value_counts()).sort_values().plot(kind='bar');

In [ ]:
#时间
(df_loss['wkd_ind'].value_counts()/df_clean['wkd_ind'].value_counts()).sort_values().plot(kind='bar');

In [ ]:
#产品
(df_loss['product'].value_counts()/df_clean['product'].value_counts()).sort_values().plot(kind='bar');

可以看出：
- 成都、广州、重庆和西安都有14%左右的亏本商品
- 线下销售亏本商品比例要高于线上渠道
- 周内销售亏本商品比例要高于周末
- 牛仔裤和运动类是亏损率最高的商品，近乎60%

In [ ]:
df_clean[['uni_revenue_of_product','unit_cost','margin']].corr()

整体来看，单位商品的销售额与成本之间是成正相关的，相关系数为0.5；单位商品的销售额与利润成强正相关，相关系数为0.91；但商品的成本与利润的相关性很低，仅为0.1.

# 总结

通篇我们使用了10座城市，共计22292条数据进行分析，得出以下结论：

- 数据主要集中在深圳、杭州、武汉三所城市
- 选择线下购买的客户远大于线上购买
- 客户中女性数量多于男性近一倍
- 客户的年龄段主要集中在20-40岁之间，此外，还有一些用户（60岁以上）可疑填写不实信息
- 客户选择周内购买的总次数要稍大于周末购买总次数，但如果按日平均下来，周末单日的次数要多于周内单日
- T恤是最畅销的产品

- 利润主要与单位商品销售额、单位顾客消费额及总销售额有关；
- 顾客数量、总销售额、订单数及商品数两两均呈强正相关关系。

销售建议：
- 选择线下购买的客户较多，可以通过公众号、微博等多做一些线上优惠券的活动，提高线上订单量，减轻线下门店服务压力，降低线下服务成本
- 多针对女性群体做专属活动

## 问题一：整体销售情况随着时间的变化是怎样的？

- 不管是周内还是周末，选择线上和线下的客户比例相当
- 相比于女生，男生有稍高的比例(+4%)选择在周末购买
- 24岁以下的两组比例相当，25岁以上组(>=60分组不予考虑)随着年龄的逐渐增长，在周内购物的比例也逐渐升高
- 就总量来看，周末两天的总量一直低于周内五天的总量
- 就均值来看，周末具有较高的单位成本，但也带来了更多的单位收入和单位利润
- 就标准差来看，周末的单位收入和单位利润更不稳定

销售建议：
- 可以选择在周内多做一些针对中老年的活动，而在周末多做一些针对年轻人及情侣的活动

## 问题二：不同产品的销售情况是怎样的？顾客偏爱哪一种购买方式？
- T恤占绝对领先地位，在销售额、销售数量和利润上都遥遥领先；当季新品和配件占据第二、三名，其中当季新品的销售额和数量较配件高，但利润较配件低；
- 毛衣、裙子和配件的平均利润最高，都在80元左右。
- 广州的顾客更偏爱在线上购买，上海、西安和重庆也有一定的线上购买比例，其余城市均为在线下购买；
- 不同性别在选择购买渠道时比例相当；
- 就年龄段来看，所有年龄段选择线上购买的比例相当，都在15%-20%之间，其中25-29岁选择线上比例最高，55-59岁选择线上比例最低。

销售建议：
- T恤的销量及利润都较高，可以针对T恤和销量较低的产品在一些时间节点适当的做一些捆绑活动

## 问题三：销售额和产品成本之间的关系怎么样？
- 成都、广州、重庆和西安都有14%左右的亏本商品
- 线下销售亏本商品比例要高于线上渠道
- 周内销售亏本商品比例要高于周末
- 牛仔裤和运动类是亏损率最高的商品，近乎60%
- 整体来看，单位商品的销售额与成本之间是成正相关的，相关系数为0.5；单位商品的销售额与利润成强正相关，相关系数为0.91；但商品的成本与利润的相关性很低，仅为0.1.

销售建议：
- 牛仔裤和运动类并不是优衣库的主打产品，而且亏损率较高，此类商品的竞品又有较好口碑（如李维斯、耐克等），所以可以减少此类商品的产量